# GEOG5995M Final Assignment (Template)

Student ID number: 

In [51]:
# read in required packages

In [52]:
# import pandas package and rename to pd
# import geopandas and rename to gpd
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import pyproj
import contextily as ctx
import seaborn as sns

import geoplot as gplt
import geoplot.crs as gcrs

This is an example block of mardown text I want to reference <a href="#ref1">[1]</a>. I might need to add some more citations <a href="#ref2">[2]</a><a href="#ref2">[3]</a>



In [71]:
# create dataframe called 'health' by reading in csv file named
health = pd.read_xlsx('/Users/user/Documents/GEOG5995M_Programming_For_Social_Sciences/Assignment/2.Final_Project/Health_data.xlsx')

AttributeError: module 'pandas' has no attribute 'read_xlsx'

In [53]:
# create dataframe called 'AHAH' by reading in the csv file named 'AHAH_V3_0.csv'
ahah = pd.read_csv('/Users/user/Documents/GEOG5995M_Programming_For_Social_Sciences/Assignment/2.Final_Project/AHAH_V3_0.csv')

In [58]:
# Check the data. Here we have 58 columns comprised of one 2011 LSOA identification column and the remaining AHAH dimension variable columns. 
# We can see from the non-null count that we have no missing data as each column has the correct number of cells matching the 41729 entries.
# We also are able to see what data type each column is. 
ahah.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41729 entries, 0 to 41728
Data columns (total 58 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   lsoa11        41729 non-null  object 
 1   ah3gp         41729 non-null  float64
 2   ah3dent       41729 non-null  float64
 3   ah3phar       41729 non-null  float64
 4   ah3hosp       41729 non-null  float64
 5   ah3blue       41729 non-null  float64
 6   ah3gpas       41729 non-null  float64
 7   ah3ffood      41729 non-null  float64
 8   ah3gamb       41729 non-null  float64
 9   ah3leis       41729 non-null  float64
 10  ah3pubs       41729 non-null  float64
 11  ah3tob        41729 non-null  float64
 12  ah3no2        41729 non-null  float64
 13  ah3so2        41729 non-null  float64
 14  ah3pm10       41729 non-null  float64
 15  ah3gp_rnk     41729 non-null  int64  
 16  ah3dent_rnk   41729 non-null  int64  
 17  ah3phar_rnk   41729 non-null  int64  
 18  ah3hosp_rnk   41729 non-nu

In [77]:
# We do not need all 58 dimension columns so we will create a new dataframe containing the dimensions we are most interested in. 
# We will need the lsoa11 column and choose the health domain score, green/bluespace domain score, air quality domain score, AHAH index score
ahah2 = ahah[['lsoa11','ah3h','ah3g','ah3e','ah3ahah']].copy()

In [79]:
ahah2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41729 entries, 0 to 41728
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   lsoa11   41729 non-null  object 
 1   ah3h     41729 non-null  float64
 2   ah3g     41729 non-null  float64
 3   ah3e     41729 non-null  float64
 4   ah3ahah  41729 non-null  float64
dtypes: float64(4), object(1)
memory usage: 1.6+ MB


In [80]:
# Read in a data file so we can match up area codes
national = pd.read_csv('/Users/user/Documents/GEOG5995M_Programming_For_Social_Sciences/Assignment/2.Final_Project/LSOA_(2011)_to_LSOA_(2021)England_and_Wales.csv')

In [81]:
# Check the data
national.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35796 entries, 0 to 35795
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   LSOA11CD  35796 non-null  object
 1   LSOA11NM  35796 non-null  object
 2   LSOA21CD  35796 non-null  object
 3   LSOA21NM  35796 non-null  object
 4   CHGIND    35796 non-null  object
 5   LAD22CD   35796 non-null  object
 6   LAD22NM   35796 non-null  object
 7   LAD22NMW  1940 non-null   object
 8   ObjectId  35796 non-null  int64 
dtypes: int64(1), object(8)
memory usage: 2.5+ MB


In [82]:
# Create a new dataframe called 'York' by only reading in 'York' data from the national dataframe's 'LAD22NM' column
york = national[(national['LAD22NM'] == 'York')]

In [83]:
# Confirm it worked by checking that the first 5 rows in the dataframe include 'York' in the LAD22NM column
york.head()

,LSOA11CD,LSOA11NM,LSOA21CD,LSOA21NM,CHGIND,LAD22CD,LAD22NM,LAD22NMW,ObjectId
12408,E01013428,York 006A,E01013428,York 006A,U,E06000014,York,NaN,12409
12415,E01013429,York 006B,E01013429,York 006B,U,E06000014,York,NaN,12416
12421,E01013430,York 006C,E01013430,York 006C,U,E06000014,York,NaN,12422
12425,E01013431,York 006D,E01013431,York 006D,U,E06000014,York,NaN,12426
12427,E01013432,York 007C,E01013432,York 007C,U,E06000014,York,NaN,12428


In [84]:
# Now to clean up the table. First look at the columns and then remove any not needed
york.info()

<class 'pandas.core.frame.DataFrame'>
Index: 121 entries, 12408 to 33147
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   LSOA11CD  121 non-null    object
 1   LSOA11NM  121 non-null    object
 2   LSOA21CD  121 non-null    object
 3   LSOA21NM  121 non-null    object
 4   CHGIND    121 non-null    object
 5   LAD22CD   121 non-null    object
 6   LAD22NM   121 non-null    object
 7   LAD22NMW  0 non-null      object
 8   ObjectId  121 non-null    int64 
dtypes: int64(1), object(8)
memory usage: 9.5+ KB


In [85]:
# Remove columns containing 2021 data and the LAD22NMW column as this does not contain any data
york2 = york.drop(['LSOA21CD','LSOA21NM','LAD22NMW'], axis = 1)

In [86]:
# Check it worked. We removed 3 columns so the total has gone down from 9 to 6.
york2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 121 entries, 12408 to 33147
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   LSOA11CD  121 non-null    object
 1   LSOA11NM  121 non-null    object
 2   CHGIND    121 non-null    object
 3   LAD22CD   121 non-null    object
 4   LAD22NM   121 non-null    object
 5   ObjectId  121 non-null    int64 
dtypes: int64(1), object(5)
memory usage: 6.6+ KB


In [88]:
# Now merge dataframes by joining ahah onto york2 (york2 being left) by using LSOA code as merger
# We should have our six columns from the york2 dataframe along with the 5 columns making up the ahah2 dataframe
york2.merge(ahah2, left_on='LSOA11CD', right_on='lsoa11')

,LSOA11CD,LSOA11NM,CHGIND,LAD22CD,LAD22NM,ObjectId,lsoa11,ah3h,ah3g,ah3e,ah3ahah
0,E01013428,York 006A,U,E06000014,York,12409,E01013428,0.298562,0.151518,-0.124521,17.085668
1,E01013429,York 006B,U,E06000014,York,12416,E01013429,-0.053671,-0.189667,-0.391200,10.014081
2,E01013430,York 006C,U,E06000014,York,12422,E01013430,0.791085,-0.489189,-0.523431,14.073578
3,E01013431,York 006D,U,E06000014,York,12426,E01013431,-0.186099,0.294428,-0.134309,14.475337
4,E01013432,York 007C,U,E06000014,York,12428,E01013432,0.143721,-0.147138,-0.342039,12.028219
...,...,...,...,...,...,...,...,...,...,...,...
116,E01033068,York 013G,U,E06000014,York,31395,E01033068,-1.309191,0.114354,-0.097985,26.857438
117,E01033069,York 013H,U,E06000014,York,31405,E01033069,-1.035699,-0.090876,-0.162730,27.469509
118,E01033070,York 013I,U,E06000014,York,31415,E01033070,-1.862147,0.072469,-0.024825,32.508178
119,E01013365,York 023A,S,E06000014,York,33147,E01013365,0.414843,-0.410215,-0.326714,13.166565


In [55]:
geoyork = gpd.read_file('/Users/user/Documents/GEOG5995M_Programming_For_Social_Sciences/Assignment/2.Final_Project/Output_Area_Classification_2011/Local_Authority_Districts/E06000014/shapefiles/E06000014.shp')

In [56]:
geoyork.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 637 entries, 0 to 636
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   oa11cd    637 non-null    object  
 1   lsoa11cd  637 non-null    object  
 2   geometry  637 non-null    geometry
dtypes: geometry(1), object(2)
memory usage: 15.1+ KB


Data Visualisation
- Don't forget to present your final two data visualisation (one spatial and one non-spatial) and the justifications about the deicsisions you made whilst preparing and visualising the data.

In [57]:
geoyork.head()

,oa11cd,lsoa11cd,geometry
0,E00067299,E01013335,"POLYGON ((457218.959 452203.985, 457210.522 45..."
1,E00067300,E01013337,"POLYGON ((456899.692 451950.658, 456898.978 45..."
2,E00067301,E01013339,"POLYGON ((457270.101 452710.928, 457272.232 45..."
3,E00067302,E01013339,"POLYGON ((456861.658 452647.570, 456853.780 45..."
4,E00067303,E01013339,"POLYGON ((457084.777 452564.766, 457084.000 45..."


## References
<p><a href="add_url_here">[1]</a>Footnote citation goes here</p>
<p><a href="https://github.com/FrancescaPontin/GEOG5995">[2]</a>GEOG5995M course materials</p>
<p><a href="add_url_here">[3]</a>Citation 3</p>